In [1]:
import pandas as pd
# import module as mod
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from logging import exception



# RECUPERATION DATA

In [2]:
data = pd.read_csv('base-joconde-extrait.csv', delimiter=';',low_memory=False ,encoding='utf-8')
data.dataframeName = 'base-joconde-extrait.csv'

In [9]:
dff=data.sample(frac=0.01)
dff

,ID-notice,POP_CONTIENT_GEOLOCALISATION,POP_COORDONNEES,Ancien dépôt,Appellation,APTN,ATTR,Auteur,BASE,BIBL,...,REGION,Sujet,RETIF,Source de la représentation,Statut juridique,Matériaux-techniques,Titre,Utilisation,WWW,Ville_
18956,00160018055,oui,"48.579654,7.736231",NaN,NaN,NaN,NaN,BALTHASAR (dit);HAUG Hans (patronyme),Collections des musées de France (Joconde),NaN,...,Grand Est,NaN,NaN,NaN,don;Strasbourg;musée d'art moderne et contempo...,NaN,Château de Cheverny,NaN,NaN,Strasbourg
117593,08680000588,oui,"43.580973,7.127959",NaN,NaN,NaN,NaN,MASEREEL Frans,Collections des musées de France (Joconde),. GIRAUDY (Danièle). - Les collections d'art m...,...,Provence-Alpes-Côte d'Azur,NaN,NaN,NaN,don;Antibes;musée Picasso,NaN,Voisins,NaN,NaN,Antibes
475111,50350022175,oui,"48.859903,2.340929",NaN,NaN,"Cosson, Paul Henri Charles#dernière provenance...",NaN,BRETON Jules,Collections des musées de France (Joconde),NaN,...,Ile-de-France,autoportrait,NaN,NaN,propriété de l'Etat;legs;musée du Louvre dépar...,plume;encre brune;mine de plomb,"Autoportrait, de profil vers la droite",NaN,http://arts-graphiques.louvre.fr/detail/oeuvre...,Paris
246051,50350202901,oui,"48.859903,2.340929",NaN,NaN,"Krahe, Lambert ; Maratta, albums dits de",ancienne attribution : ANONYME ITALIEN fin XVI...,CALANDRUCCI Giacinto,Collections des musées de France (Joconde),NaN,...,Ile-de-France,NaN,NaN,NaN,propriété de l'Etat;musée du Louvre départemen...,sanguine;blanc (rehaut);papier (bleu),"Religieuse priant agenouillée, reprises des ma...",NaN,http://arts-graphiques.louvre.fr/detail/oeuvre...,Paris
526223,50190041936,oui,"45.861683,5.947732",NaN,NaN,NaN,NaN,HUMBERT Jules Eugène;manufacture de Sèvres,Collections des musées de France (Joconde),NaN,...,Ile-de-France,NaN,NaN,NaN,propriété de l'Etat;archives publiques;Sèvres ...,"graphite, lavis;papier",Elément décoratif [La Nature],NaN,NaN,Sèvres
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102830,70500002620,oui,"43.701924,7.279572",NaN,NaN,Jean-Baptiste Barla,NaN,BARLA Jean-Baptiste (attribué à),Collections des musées de France (Joconde),Annales du Muséum d'Histoire Naturelle de Nice...,...,Provence-Alpes-Côte d'Azur,représentation scientifique (champignon),NaN,NaN,propriété de la commune;legs;Nice;muséum d'his...,"papier, encre, aquarelle",Lentin écailleux ; champignon,NaN,NaN,Nice
436718,5001MIL3645,non,"48.897879,2.094936",NaN,NaN,De Baye,NaN,NaN,Collections des musées de France (Joconde),NaN,...,Ile-de-France,NaN,NaN,NaN,propriété de l'Etat;don;musée d'Archéologie na...,silex,NaN,NaN,NaN,Saint-Germain-en-Laye
578961,M0332012969,oui,"47.240055,6.022938",NaN,NaN,Jean Gigoux,LE BRUN Charles ; anonyme,anonyme,Collections des musées de France (Joconde),NaN,...,Bourgogne-Franche-Comté,"portrait d'inconnu (homme, en buste)",NaN,NaN,propriété de la commune;legs;Besançon;musée de...,"papier, pierre noire, craie blanche",Portrait d'homme,NaN,NaN,Besançon
66897,01720012013,oui,"46.30716,4.832892",NaN,NaN,NaN,NaN,NaN,Collections des musées de France (Joconde),NaN,...,Bourgogne-Franche-Comté,NaN,NaN,NaN,propriété de la commune;mode d'acquisition inc...,noyer,NaN,NaN,NaN,Mâcon


In [6]:
import joblib as jl

In [11]:
jl.dump(dff,'dff.jl')

['dff.jl']

In [91]:
df=data.copy()
df.shape

(650853, 75)

## reduction de colonne

In [92]:
# % de remplissage des colonnes
na=df.isnull().sum()/df.shape[0]*100
na=na.sort_values()
na

ID-notice                         0.000000
POP_CONTIENT_GEOLOCALISATION      0.000000
POP_COORDONNEES                   0.000000
Statut juridique                  0.000000
PRODUCTEUR                        0.000000
                                   ...    
MANQUANT_COM                     99.830837
REFPAL                           99.997081
REFMER                           99.999846
REFMEM                           99.999846
NSDA                            100.000000
Length: 75, dtype: float64

In [93]:
# Suppression des colonne supperieur a 50% de NAN
na50=na[na.values<50].index
df=df[na50]

In [94]:
df.columns

Index(['ID-notice', 'POP_CONTIENT_GEOLOCALISATION', 'POP_COORDONNEES',
       'Statut juridique', 'PRODUCTEUR', 'Numéro de l'objet', 'BASE',
       'CONTIENT_IMAGE', 'Domaine', 'Lieu de conservation', 'Ville_',
       'Identifiant Museofile', 'Source et date de la notice', 'DPT', 'NOMOFF',
       'REGION', 'IMAGE', 'Label Musée de France', 'Matériaux-techniques',
       'Dimensions', 'Date d'import', 'LOCA2', 'Période de création', 'Auteur',
       'Date d'acquisition', 'Titre', 'APTN', 'PHOT', 'Dénomination',
       'Précisions sur l'auteur', 'Ecole', 'Date de mise à jour', 'Sujet',
       'MSGCOM'],
      dtype='object')

In [95]:
#choix des colonnes utiles
col = ['ID-notice',
    'Auteur',
    'Titre', 
    'Dénomination',
    'Domaine', 
    'Sujet',
    # 'POP_CONTIENT_GEOLOCALISATION', 
    # 'Statut juridique',
    # 'PRODUCTEUR',
    # "Numéro de l'objet",
    'BASE',
    'CONTIENT_IMAGE', 
    # 'Lieu de conservation', 
    'Identifiant Museofile', 
    # 'Source et date de la notice', 
    'NOMOFF',
    'LOCA2', 
    'REGION', 
    'DPT', 
    'Ville_',
    'POP_COORDONNEES',
    # 'IMAGE', 
    # 'Label Musée de France', 
    # 'Matériaux-techniques',
    # 'Dimensions', 
    # "Date d'import", 
    # 'Période de création', 
    # "Date d'acquisition", 
    # 'APTN', 
    # 'PHOT', 
    # "Précisions sur l'auteur", 
    # 'Ecole', 
    # 'Date de mise à jour', 
    # 'MSGCOM'
]
len(col)

15

In [96]:
df=df[col]
df.shape

(650853, 15)

## clean encodage

- Il y a des lignes qui sont mal encodées
- la colonne base ne contient qu'une valeur. Mais elle en contient des mal encodé. ca nous permetra de recuperer les lignes mal encodées

In [97]:
# recup de la valeur base mal encodé ( c'est le string le plus long)

a=list(df.BASE.unique())
a.sort(key = len, reverse=True)
wrong_base = a[0]

# creation de 2 db , dont 1 mal encodé
df_cp1252=df[df.BASE == wrong_base]
df_ok=df[df.BASE == a[1]]


In [98]:
# verification qu' on recupere bien toutes les données
print(df_cp1252.shape + df_ok.shape)
df_cp1252.shape[0] + df_ok.shape[0]

(43333, 15, 607520, 15)


650853

In [99]:
df_cp1252.head(2)

,ID-notice,Auteur,Titre,Dénomination,Domaine,Sujet,BASE,CONTIENT_IMAGE,Identifiant Museofile,NOMOFF,LOCA2,REGION,DPT,Ville_,POP_COORDONNEES
177945,00660019655,LACOUR (FILS) Pierre (d'aprÃ¨s le dessin de);M...,NaN,assiette,céramique;arts décoratifs,"groupe de figure (nourrice, enfant, madras, fl...",Collections des musÃ©es de France (Joconde),oui,M0066,musÃ©e des arts dÃ©coratifs et du design,NaN,Nouvelle-Aquitaine,Gironde,Bordeaux,"44.838956,-0.579686"
177948,00660019661,Manufacture David Johnston,La chasse au canard,assiette,céramique;arts décoratifs;chasse - pÃªche - cu...,"scÃ¨ne (chasseur, chien, canard, chasse)",Collections des musÃ©es de France (Joconde),oui,M0066,musÃ©e des arts dÃ©coratifs et du design,NaN,Nouvelle-Aquitaine,Gironde,Bordeaux,"44.838956,-0.579686"


In [100]:
df_cp1252.columns

Index(['ID-notice', 'Auteur', 'Titre', 'Dénomination', 'Domaine', 'Sujet',
       'BASE', 'CONTIENT_IMAGE', 'Identifiant Museofile', 'NOMOFF', 'LOCA2',
       'REGION', 'DPT', 'Ville_', 'POP_COORDONNEES'],
      dtype='object')

In [101]:
def convert(s):
    if s is not None:
        try:
            if isinstance(s,str):
                s = s.encode("cp1252").decode()
            else : 
                s = str(s).encode("cp1252").decode()
                
        except UnicodeError:
            pass
    return s

columns = ['Auteur','Titre', 'Dénomination', 'Domaine', 'Sujet','BASE','NOMOFF', 'LOCA2','REGION', 'DPT', 'Ville_' ]
for column in columns:
    df_cp1252.loc[:, column] = df[column].apply(convert)

C:\Users\e_goy\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


In [102]:
# concatenation des 2 df
df1 = pd.concat([df_cp1252, df_ok])
df1.shape

(650853, 15)

## Suppression des lignes

In [103]:
# Ligne en double
df1 = df1.drop_duplicates(subset=['ID-notice'],keep='first')
df1.shape

(639325, 15)

In [104]:
# Oeuvre n'ayant pas de musé 
df1 = df1.dropna(subset=['Identifiant Museofile'])

In [105]:
#suppression des nan Identifiant Museofile ne contiene aune info  de locallisation
df1 = df1.loc[df1['Identifiant Museofile'].dropna().index]

print(df1.shape)
df1.head(3)

(639247, 15)


,ID-notice,Auteur,Titre,Dénomination,Domaine,Sujet,BASE,CONTIENT_IMAGE,Identifiant Museofile,NOMOFF,LOCA2,REGION,DPT,Ville_,POP_COORDONNEES
177945,00660019655,LACOUR (FILS) Pierre (d'après le dessin de);Ma...,nan,assiette,céramique;arts décoratifs,"groupe de figure (nourrice, enfant, madras, fl...",Collections des musées de France (Joconde),oui,M0066,musée des arts décoratifs et du design,nan,Nouvelle-Aquitaine,Gironde,Bordeaux,"44.838956,-0.579686"
177948,00660019661,Manufacture David Johnston,La chasse au canard,assiette,céramique;arts décoratifs;chasse - pÃªche - cu...,"scène (chasseur, chien, canard, chasse)",Collections des musées de France (Joconde),oui,M0066,musée des arts décoratifs et du design,nan,Nouvelle-Aquitaine,Gironde,Bordeaux,"44.838956,-0.579686"
177951,01610014524,nan,nan,console (2),ameublement;sculpture,"(console, applique)",Collections des musées de France (Joconde),oui,M0161,musée Rolin,nan,Bourgogne-Franche-Comté,Saône-et-Loire,Autun,"46.945723,4.299216"


In [106]:
# Mettre des NAN
df1.replace('nan',np.nan, inplace=True)
df1.replace('',np.nan, inplace=True)


## Recup auteur

In [107]:

def nb_autor(aut):
    b=1
    if ';' in aut:
        b=len(aut.split(";"))
    return b
        

In [108]:
def normalize(auts):
    clean_auts=[]
    for aut in auts : 
        if '(' in aut:
            aut=aut.split('(')[0]
        if aut:
            clean_auts.append(aut.lower())

    return  clean_auts

In [109]:
from pandas import isna


def net_aut(aut):
    if not isna(aut) :
        if not isinstance(aut, str):
            aut=str(aut)
            
        if ';' in aut:
            aut = aut.split(';')
        else:
            aut=[aut]

        aut=normalize(aut) 
        
    return aut


In [110]:
# nettoyage des auteur : split , lower , et garder juste le nom/prenom

df1.Auteur=df1.Auteur.apply(net_aut)

In [111]:
# df_aut = df1[['Auteur','ID-notice']]
# df_aut=df_aut.set_index('ID-notice')
# df_aut

In [112]:
lst=df1.Auteur[df1.Auteur.notna()].values.tolist()

In [113]:
aut_lst = []
for auts in lst:
    aut_lst.extend(auts)
aut_lst=list(set(aut_lst))

In [114]:
len(aut_lst)

55015

## Recup musée

In [115]:
df2=df1.copy()

In [116]:
df2['Identifiant Museofile'].unique().shape

(488,)

In [117]:
df2 = df2[['Identifiant Museofile','NOMOFF','REGION','DPT','Ville_','POP_COORDONNEES']]
df2.columns = ['id','nom','region','dept','ville','geoloc']

In [118]:
df2

,id,nom,region,dept,ville,geoloc
177945,M0066,musée des arts décoratifs et du design,Nouvelle-Aquitaine,Gironde,Bordeaux,"44.838956,-0.579686"
177948,M0066,musée des arts décoratifs et du design,Nouvelle-Aquitaine,Gironde,Bordeaux,"44.838956,-0.579686"
177951,M0161,musée Rolin,Bourgogne-Franche-Comté,Saône-et-Loire,Autun,"46.945723,4.299216"
177954,M0161,musée Rolin,Bourgogne-Franche-Comté,Saône-et-Loire,Autun,"46.945723,4.299216"
177957,M0161,musée Rolin,Bourgogne-Franche-Comté,Saône-et-Loire,Autun,"46.945723,4.299216"
...,...,...,...,...,...,...
650185,M0805,Maison Gallé-Juillet,Hauts-de-France,Oise,Creil,"49.260793,2.473388"
650186,M0189,musée municipal,Bourgogne-Franche-Comté,Yonne,Sens,"48.197828,3.284505"
650187,M0161,musée Rolin,Bourgogne-Franche-Comté,Saône-et-Loire,Autun,"46.945723,4.299216"
650188,M0161,musée Rolin,Bourgogne-Franche-Comté,Saône-et-Loire,Autun,"46.945723,4.299216"


In [119]:
df2.drop(df2[df2['id']== '0000'].index,inplace=True)
df2['id'].replace('5027','M5027',inplace=True)
df2['id']=df2['id'].apply(lambda x: x.upper())

In [120]:
# met en majuscule le Mdevant le numero de musé


In [121]:
#ajoute un M a ceux qui ne l ont pas
# df2['Identifiant Museofile']=df2['Identifiant Museofile'].apply(lambda x: 'M'+x if x[0]!='M' else x)

In [122]:
# ordonne ceux qui ont le plus de données en premier et supprime les doublons
df2['nb_nan']=df2.isna().sum(axis=1)
df2=df2.sort_values('nb_nan')
df2=df2.drop_duplicates(subset='id',keep='first')
df2=df2.drop(columns=['nb_nan'])
df2=df2.sort_values('id')

In [123]:
df2['id'].unique().shape



(485,)

In [124]:
df2=df2.reset_index(drop=True)
df2=df2.fillna(['inconnue'])
df2

TypeError: "value" parameter must be a scalar or dict, but you passed a "list"

In [ ]:
df2.iloc[0,:]

id                                        B9001
nom       Bibliothèque de l’Assemblée Nationale
region                            Ile-de-France
dept                                      75007
ville                Paris, Assemblée nationale
geoloc                         48.86215,2.31897
Name: 549015, dtype: object

In [ ]:
line=df2.iloc[0,:].values.tolist()
line

['B9001',
 'Bibliothèque de l’Assemblée Nationale',
 'Ile-de-France',
 '75007',
 'Paris, Assemblée nationale',
 '48.86215,2.31897']

In [ ]:
for i in df2[:5]:
    print (i)


id
nom
region
dept
ville
geoloc


In [ ]:
a=np.nan

In [ ]:
np.isnan(a)

True

In [ ]:
df2


,id,nom,region,dept,ville,geoloc
549015,B9001,Bibliothèque de l’Assemblée Nationale,Ile-de-France,75007,"Paris, Assemblée nationale","48.86215,2.31897"
387237,B9002,Bibliothèque municipale de Grasse,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Grasse,"43.65559,6.91736"
388838,B9003,Bibliothèque municipale de Besançon,Bourgogne-Franche-Comté,Doubs,Besançon,"47.23649,6.0274"
635366,M0000,NaN,NaN,NaN,Troyes,"0.0,0.0"
426047,M0003,musée du Pays de Hanau - histoire et vies d’un...,Grand Est,Bas-Rhin,Bouxwiller,"48.824977,7.482915"
...,...,...,...,...,...,...
446855,M7048,muséum-Aquarium de Nancy,Grand Est,Meurthe-et-Moselle,Nancy,"48.695072,6.188467"
385579,M7050,muséum d'histoire naturelle,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Nice,"43.701924,7.279572"
547701,M7051,muséum d'histoire naturelle et de préhistoire,Occitanie,Gard,Nîmes,"43.83798099757179,4.363712139325677"
610512,M9007,musée de l’artillerie et de l’Infanterie,Provence-Alpes-Côte d'Azur,Var,Versailles,"43.529648,6.495058"


In [ ]:
df.Auteur

0                            LE GULUCHE Joseph Marie
1                                   THOMSEN Constant
2                   FJERDINGSTAD Christian (orfèvre)
3                                      DUTHOIT Louis
4                         DUTHOIT Aimé;DUTHOIT Louis
                             ...                    
650848    SAINT-GERMAIN Marguerite (De) (cÃ©ramiste)
650849                Christofle Charles (1805-1863)
650850              VERRERIE / FRANCE (cristallerie)
650851                LAMOISSE EugÃ¨ne (dessinateur)
650852                                       anonyme
Name: Auteur, Length: 650853, dtype: object

In [ ]:
# df.Auteur = df.Auteur.apply(lambda x: ['non renseigné'] if pd.isnull(x) else x)

In [ ]:
df[df.Auteur.isna()]

,ID-notice,Auteur,Titre,Dénomination,Domaine,Sujet,BASE,CONTIENT_IMAGE,Identifiant Museofile,NOMOFF,LOCA2,REGION,DPT,Ville_,POP_COORDONNEES


In [ ]:
a=['qsg',]
if pd.isnull(a).any() :
    print('notok')

In [ ]:
b=np.nan
pd.isnull(b).all()

AttributeError: 'bool' object has no attribute 'all'

In [ ]:
df.Auteur.isna().sum()

0

In [125]:
df3=df.copy()

In [129]:
df3.Auteur.fillna('non renseigné')
df3.Auteur.replace(np.nan,['non renseigné'])

TypeError: Invalid "to_replace" type: 'float'

In [128]:
df3[df3.Auteur.isna()]

,ID-notice,Auteur,Titre,Dénomination,Domaine,Sujet,BASE,CONTIENT_IMAGE,Identifiant Museofile,NOMOFF,LOCA2,REGION,DPT,Ville_,POP_COORDONNEES
33,M0794014836,NaN,NaN,briquet,ethnologie;vie militaire,NaN,Collections des musées de France (Joconde),oui,M0794,musée municipal,Picardie ; Aisne,Hauts-de-France,Aisne,Soissons,"49.384539,3.327233"
34,M0794014788,NaN,NaN,briquet,ethnologie;vie militaire,"figure (homme, George V d'Angleterre);figure",Collections des musées de France (Joconde),oui,M0794,musée municipal,Picardie ; Aisne,Hauts-de-France,Aisne,Soissons,"49.384539,3.327233"
35,M0794014744,NaN,NaN,coupe-papier,ethnologie;vie militaire,NaN,Collections des musées de France (Joconde),oui,M0794,musée municipal,Picardie ; Aisne,Hauts-de-France,Aisne,Soissons,"49.384539,3.327233"
36,M0794014741,NaN,NaN,coupe-papier,ethnologie;vie militaire,NaN,Collections des musées de France (Joconde),oui,M0794,musée municipal,Picardie ; Aisne,Hauts-de-France,Aisne,Soissons,"49.384539,3.327233"
87,01890010511,NaN,NaN,lame,archéologie;néolithique;industrie lithique,NaN,Collections des musées de France (Joconde),oui,M0189,musée municipal,Bourgogne ; Yonne,Bourgogne-Franche-Comté,Yonne,Sens,"48.197826,3.282739"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650833,50520012333,NaN,NaN,fauteuil,ameublement,NaN,Collections des musÃ©es de France (Joconde),oui,M5052,musÃ©e CondÃ©,NaN,Hauts-de-France,Oise,Chantilly,"49.19397,2.48522"
650834,50520012361,NaN,"Petite table en acajou, un tiroir, deux boutons",NaN,ameublement,NaN,Collections des musÃ©es de France (Joconde),non,M5052,musÃ©e CondÃ©,NaN,Hauts-de-France,Oise,Chantilly,"49.19397,2.48522"
650835,50520012322,NaN,"Fauteuil avec palmettes aux accoudoirs, recouv...",fauteuil,ameublement,NaN,Collections des musÃ©es de France (Joconde),oui,M5052,musÃ©e CondÃ©,NaN,Hauts-de-France,Oise,Chantilly,"49.19397,2.48522"
650836,50520012821,NaN,Chaises Ã pieds tournÃ©s et Ã roulettes reco...,chaise,ameublement,NaN,Collections des musÃ©es de France (Joconde),non,M5052,musÃ©e CondÃ©,NaN,Hauts-de-France,Oise,Chantilly,"49.19397,2.48522"
